In [1]:
import pyspark.sql as sparksession
from pyspark.sql.functions import *
import pandas as pd
spark = sparksession.SparkSession.builder.appName('Coffeshop').getOrCreate()



24/11/06 12:54:02 WARN Utils: Your hostname, MacBook-Pro.local resolves to a loopback address: 127.0.0.1; using 10.87.132.44 instead (on interface en0)
24/11/06 12:54:02 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/11/06 12:54:03 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [2]:
#read the csv file
df=spark.read.csv("./data/Coffee Shop Sales.csv",header=True)
                
        

In [3]:
df.show(5)

+--------------+----------------+----------------+---------------+--------+---------------+----------+----------+------------------+--------------------+--------------------+
|transaction_id|transaction_date|transaction_time|transaction_qty|store_id| store_location|product_id|unit_price|  product_category|        product_type|      product_detail|
+--------------+----------------+----------------+---------------+--------+---------------+----------+----------+------------------+--------------------+--------------------+
|             1|      01/01/2023|        07:06:11|              2|       5|Lower Manhattan|        32|         3|            Coffee|Gourmet brewed co...|         Ethiopia Rg|
|             2|      01/01/2023|        07:08:56|              2|       5|Lower Manhattan|        57|       3.1|               Tea|     Brewed Chai tea|Spicy Eye Opener ...|
|             3|      01/01/2023|        07:14:04|              2|       5|Lower Manhattan|        59|       4.5|Drinking Cho

In [4]:
#check is there any null value in the data
df.select([sum(when(col(c).isNull(), 1).otherwise(0)).alias(c) for c in df.columns]) 

DataFrame[transaction_id: bigint, transaction_date: bigint, transaction_time: bigint, transaction_qty: bigint, store_id: bigint, store_location: bigint, product_id: bigint, unit_price: bigint, product_category: bigint, product_type: bigint, product_detail: bigint]

In [5]:
df.printSchema()

root
 |-- transaction_id: string (nullable = true)
 |-- transaction_date: string (nullable = true)
 |-- transaction_time: string (nullable = true)
 |-- transaction_qty: string (nullable = true)
 |-- store_id: string (nullable = true)
 |-- store_location: string (nullable = true)
 |-- product_id: string (nullable = true)
 |-- unit_price: string (nullable = true)
 |-- product_category: string (nullable = true)
 |-- product_type: string (nullable = true)
 |-- product_detail: string (nullable = true)



In [6]:
#change the data type of the columns
df = df.withColumn('transaction_date', to_date(df['transaction_date'], 'dd/MM/yyyy'))
df=df.withColumn('transaction_qty', col('transaction_qty').cast('int'))
df=df.withColumn('store_id', col('store_id').cast('int'))
df=df.withColumn('product_id', col('product_id').cast('int'))
df=df.withColumn('unit_price', col('unit_price').cast('float'))
df=df.withColumn('transaction_id', col('transaction_id').cast('int'))


In [7]:
df.printSchema()

root
 |-- transaction_id: integer (nullable = true)
 |-- transaction_date: date (nullable = true)
 |-- transaction_time: string (nullable = true)
 |-- transaction_qty: integer (nullable = true)
 |-- store_id: integer (nullable = true)
 |-- store_location: string (nullable = true)
 |-- product_id: integer (nullable = true)
 |-- unit_price: float (nullable = true)
 |-- product_category: string (nullable = true)
 |-- product_type: string (nullable = true)
 |-- product_detail: string (nullable = true)



In [8]:
#count the number of rows in the dataframe
print('row number',df.count(),' column number',len(df.columns))

row number 149116  column number 11


In [9]:
# find dupplicate rows
duplicate_count=df.groupBy(df.columns).count().filter("count>1").agg({"count":"sum"}).collect()[0][0]
print(duplicate_count)

None


In [10]:
df.createOrReplaceTempView('coffeeshop')

In [17]:
#which days of the week are the busiest?
spark.sql("""
          SELECT date_format(transaction_date, 'EEEE') as day , round(sum(transaction_qty*unit_price),2) as total_sales FROM coffeeshop
          group by 1
          order by total_sales desc
          """).show()

+---------+-----------+
|      day|total_sales|
+---------+-----------+
|   Monday|  101677.28|
|   Friday|   101373.0|
| Thursday|  100767.78|
|Wednesday|  100313.54|
|  Tuesday|   99455.94|
|   Sunday|   98330.31|
| Saturday|   96894.48|
+---------+-----------+



In [19]:
#which store location has the highest sales?
spark.sql("""
          SELECT store_location, round(sum(transaction_qty*unit_price),2) as total_sales FROM coffeeshop
          group by store_location
          order by total_sales desc
          """).show()

+---------------+-----------+
| store_location|total_sales|
+---------------+-----------+
| Hell's Kitchen|  236511.17|
|        Astoria|  232243.91|
|Lower Manhattan|  230057.25|
+---------------+-----------+



In [21]:
#which store location's busiest days ?
spark.sql("""
          SELECT store_location,date_format(transaction_date, 'EEEE') as day , round(sum(transaction_qty*unit_price),2) as total_sales FROM coffeeshop
          group by store_location,day
          order by store_location,total_sales desc
          """).show()

+---------------+---------+-----------+
| store_location|      day|total_sales|
+---------------+---------+-----------+
|        Astoria|Wednesday|   34244.63|
|        Astoria| Thursday|   34140.37|
|        Astoria|   Monday|   33928.29|
|        Astoria|   Friday|   33472.75|
|        Astoria|   Sunday|   32795.18|
|        Astoria| Saturday|   31845.93|
|        Astoria|  Tuesday|   31816.76|
| Hell's Kitchen|  Tuesday|   34846.47|
| Hell's Kitchen|   Friday|   34743.18|
| Hell's Kitchen|Wednesday|   33779.09|
| Hell's Kitchen|   Sunday|   33685.63|
| Hell's Kitchen| Thursday|   33468.06|
| Hell's Kitchen|   Monday|   33389.51|
| Hell's Kitchen| Saturday|   32599.23|
|Lower Manhattan|   Monday|   34359.48|
|Lower Manhattan| Thursday|   33159.35|
|Lower Manhattan|   Friday|   33157.07|
|Lower Manhattan|  Tuesday|   32792.71|
|Lower Manhattan| Saturday|   32449.32|
|Lower Manhattan|Wednesday|   32289.82|
+---------------+---------+-----------+
only showing top 20 rows



In [40]:
#what are the most sold 3 product category per store?
spark.sql("""
         SELECT store_location, product_category ,total_sales from
          (
         SELECT store_location, product_category , round(sum(transaction_qty*unit_price),2) as total_sales 
          , rank() over(partition by store_location order by sum(transaction_qty*unit_price) desc) as rank
            FROM coffeeshop
          group by store_location, product_category
          order by store_location,total_sales desc) as subquery
          where rank <=3
          
          """).show()

+---------------+----------------+-----------+
| store_location|product_category|total_sales|
+---------------+----------------+-----------+
|        Astoria|          Coffee|    89744.3|
|        Astoria|             Tea|    67839.9|
|        Astoria|          Bakery|   26599.75|
| Hell's Kitchen|          Coffee|   91222.65|
| Hell's Kitchen|             Tea|    64701.3|
| Hell's Kitchen|          Bakery|   27386.95|
|Lower Manhattan|          Coffee|    88985.5|
|Lower Manhattan|             Tea|   63864.75|
|Lower Manhattan|          Bakery|   28328.94|
+---------------+----------------+-----------+



In [41]:
#what are the most sold 3 product per store?
spark.sql("""
         SELECT store_location, product_detail ,total_sales from
          (
         SELECT store_location, product_detail , round(sum(transaction_qty*unit_price),2) as total_sales 
          , rank() over(partition by store_location order by sum(transaction_qty*unit_price) desc) as rank
            FROM coffeeshop
          group by store_location, product_detail
          order by store_location,total_sales desc) as subquery
          where rank <=3
          
          """).show()

+---------------+--------------------+-----------+
| store_location|      product_detail|total_sales|
+---------------+--------------------+-----------+
|        Astoria|   Dark chocolate Lg|     7897.5|
|        Astoria|Sustainably Grown...|    7509.75|
|        Astoria|            Latte Rg|     6358.0|
| Hell's Kitchen|           Civet Cat|     7380.0|
| Hell's Kitchen|Sustainably Grown...|    7329.25|
| Hell's Kitchen|   Dark chocolate Lg|     6534.0|
|Lower Manhattan|   Dark chocolate Lg|     6574.5|
|Lower Manhattan|            Latte Rg|    6387.75|
|Lower Manhattan|Sustainably Grown...|    6312.75|
+---------------+--------------------+-----------+



In [42]:
#what are the least sold 3 product per store?
spark.sql("""
         SELECT store_location, product_detail ,total_sales from
          (
         SELECT store_location, product_detail , round(sum(transaction_qty*unit_price),2) as total_sales 
          , rank() over(partition by store_location order by sum(transaction_qty*unit_price) asc) as rank
            FROM coffeeshop
          group by store_location, product_detail
          order by store_location,total_sales desc) as subquery
          where rank <=3
          
          """).show()

+---------------+--------------------+-----------+
| store_location|      product_detail|total_sales|
+---------------+--------------------+-----------+
|        Astoria|Traditional Blend...|     277.45|
|        Astoria|           Earl Grey|      268.5|
|        Astoria|      Dark chocolate|       83.2|
| Hell's Kitchen|  Serenity Green Tea|      407.0|
| Hell's Kitchen|Spicy Eye Opener ...|     383.25|
| Hell's Kitchen|I Need My Bean! D...|      360.0|
|Lower Manhattan|          Peppermint|     331.15|
|Lower Manhattan|         Lemon Grass|      268.5|
|Lower Manhattan|      Dark chocolate|      249.6|
+---------------+--------------------+-----------+



In [71]:
#what are the most expensive 3 products sold?
spark.sql("""
        SELECT  distinct product_detail , unit_price,rank
          FROM (
         SELECT   product_detail , unit_price  , dense_rank() over (order by unit_price desc) as rank
         FROM coffeeshop
         ORDER BY  unit_price desc
          ) as subquery
        where rank <= 3
          
          
          """).show()

24/11/06 13:41:17 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
24/11/06 13:41:17 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
24/11/06 13:41:17 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
24/11/06 13:41:17 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.


+--------------------+----------+----+
|      product_detail|unit_price|rank|
+--------------------+----------+----+
|           Civet Cat|      45.0|   1|
| Organic Decaf Blend|      28.0|   2|
|I Need My Bean! T...|      28.0|   2|
|I Need My Bean! L...|      23.0|   3|
|I Need My Bean! D...|      23.0|   3|
| Organic Decaf Blend|      23.0|   3|
|I Need My Bean! T...|      23.0|   3|
+--------------------+----------+----+



In [72]:
#what are the most sold 3 products ?
spark.sql("""
        SELECT   product_detail , total_count,rank
          FROM (
         SELECT   product_detail , sum(transaction_qty) as total_count  , dense_rank() over (order by sum(transaction_qty) desc) as rank
         FROM coffeeshop
         GROUP BY product_detail
          ) as subquery
        where rank <= 3
          
          
          """).show()

24/11/06 13:44:36 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
24/11/06 13:44:36 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
24/11/06 13:44:36 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.


+--------------------+-----------+----+
|      product_detail|total_count|rank|
+--------------------+-----------+----+
|        Earl Grey Rg|       4708|   1|
|   Dark chocolate Lg|       4668|   2|
|Morning Sunrise C...|       4643|   3|
+--------------------+-----------+----+



24/11/06 13:44:36 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
24/11/06 13:44:36 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
24/11/06 13:44:36 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
24/11/06 13:44:36 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
